Needs 3 text files with data, dataset.py, dataloader.py, and models (feedforward.py)

In [ ]:
! pip install --quiet "torchvision" "torch>=1.6, <1.9" "torchmetrics>=0.3" "ipython[notebook]" "pytorch-lightning>=1.3" "torchtext"
! pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'r1.6.1'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml

In [ ]:
try:
    print("Initilaising in colab set-up...")
    from google.colab import drive
    drive.mount('/content/drive')

    import os
    os.chdir('/content/drive/MyDrive/DL_Project/')
    tpu_cores = 8
    gpu_cores = None
except(ModuleNotFoundError):
    print('Initialising non-colab set-up...')
    tpu_cores=None
    gpu_cores=None

In [ ]:
import VGGnet
from dataloader import *
import pytorch_lightning.loggers as pl_loggers
import pytorch_lightning as pl

In [ ]:
train = VoxDataset('./minidata/spectrograms/', 'train')
valid = VoxDataset('./minidata/spectrograms/', 'validation')
test = VoxDataset('./minidata/spectrograms/', 'test')

dataloader = VoxDataloader(train, valid, test, batch_size=32)

num_classes = len(train.dataset['id_int'].unique())

print(len(train))
print(len(valid))
print(len(test))

In [ ]:
model = VGGnet.VGGnet(num_classes=num_classes, lr=1e-3, batch_norm=True)
tb_logger = pl_loggers.TensorBoardLogger('./Logs/', name="BatchNorm_1e-3")
trainer = pl.Trainer(logger=tb_logger, max_epochs=100, tpu_cores=tpu_cores, gpus=gpu_cores, log_every_n_steps=20)
trainer.fit(model, dataloader)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./Logs2/
